# Tutorial — Proposições da Câmara: da API ao corpus de PDFs (com **Python**)

Neste tutorial, voltado às pessoas já habituadas à linguagem nos primeiros encontros, vamos trabalhar com dados de proposições de Projeto de Lei na Câmara dos Deputados, criando um corpus a partir dos dados obtidos na API da Câmara, passando pelo download dos arquivos até a organização de um corpus.

> **Resumo do que vamos fazer**
>
> 1. Consultar a API da Câmara para listar os **PLs de 2025**;
> 2. Buscar os metadados de cada proposição e obter os **id** e endereços dos arquivos originais das proposições de inteiro teor;
> 3. **Baixar** os PDFs de inteiro teor;
> 4. Construir um **corpus** a partir dos pdfs para análises textuais.

---


## 0) Preparação do ambiente

Vamos começar carregando os pacotes que vamos utilizar. Se estiver rodando no seu ambiente local, vale instalar as dependências (apenas na primeira vez):

```bash
pip install requests pandas pdfminer.six PyPDF2 nltk unidecode
```


In [ ]:
# imports principais
import os
import time
import json
import requests
import pandas as pd
from pathlib import Path

# leitura de PDFs
from pdfminer.high_level import extract_text  # para extrair o texto
from PyPDF2 import PdfReader                  # para contar páginas

# NLP util
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

# downloads de recursos do NLTK (somente se ainda não tiver)
try:
    _ = stopwords.words('portuguese')
except LookupError:
    nltk.download('stopwords')
    _ = stopwords.words('portuguese')

# conferir versões rápidas
pd.__version__


---

## 1) API da Câmara dos Deputados — obtendo as proposições de PLs de 2025

Nosso primeiro objetivo é entender como funciona uma API. Antes de avançar, visite a página da [API de Dados Abertos da Câmara dos Deputados](https://dadosabertos.camara.leg.br/swagger/api.html).

Para obter os dados das proposições de PLs de 2025, vamos utilizar o endpoint `/proposicoes` com os parâmetros de sigla do tipo de proposição, ano e mais alguns parâmetros obrigatórios. Vamos limitar a 100 itens.

A API da Câmara é bem simples de usar em Python: vamos construir uma requisição HTTP com `requests.get`, passar os parâmetros em `params` e, no retorno JSON, acessar o campo `dados` com as informações básicas das proposições.


In [ ]:
BASE = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"

params = {
    "siglaTipo": "PL",
    "ano": 2025,
    "ordem": "ASC",
    "ordenarPor": "id",
    "itens": 100
}

resp = requests.get(BASE, headers={"Accept": "application/json"}, params=params)
resp.raise_for_status()
dados = resp.json().get("dados", [])
df_proposicoes_simples = pd.DataFrame(dados)
df_proposicoes_simples.head()


> **Nota rápida sobre paginação da API**: se você quiser **mais que 100** registros, inclua o parâmetro `pagina` em loop e agregue os resultados.  

O resultado do processo é um `DataFrame` que contém uma informação essencial para o próximo passo, que é o **id** da proposição. Vamos agora utilizar estes ids em outro endpoint da API para obter informações detalhadas das proposições.


---

## 2) Função para buscar o **detalhe** de uma proposição

Agora, para cada `id` da etapa anterior, buscamos o detalhe no endpoint `/proposicoes/{id}`. A função abaixo retorna um dicionário “achatado” com campos úteis, incluindo o **`urlInteiroTeor`**, que é o endereço que contém o documento da proposição.


In [ ]:
from typing import Optional, Dict, Any

def obter_proposicao_detalhe(id: int) -> Optional[Dict[str, Any]]:
    url = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id}"
    r = requests.get(url, headers={"Accept": "application/json"})
    if not r.ok:
        return None
    proposicao = r.json().get("dados", {})

    status_proposicao = proposicao.get("statusProposicao") or {}

    return {
        "id": proposicao.get("id"),
        "siglaTipo": proposicao.get("siglaTipo"),
        "numero": proposicao.get("numero"),
        "ano": proposicao.get("ano"),
        "descricaoTipo": proposicao.get("descricaoTipo"),
        "ementa": proposicao.get("ementa"),
        "ementaDetalhada": proposicao.get("ementaDetalhada"),
        "keywords": proposicao.get("keywords"),
        "dataApresentacao": proposicao.get("dataApresentacao"),
        "urlInteiroTeor": proposicao.get("urlInteiroTeor"),
        "uriAutores": proposicao.get("uriAutores"),
        # status
        "status_dataHora": status_proposicao.get("dataHora"),
        "status_sequencia": status_proposicao.get("sequencia"),
        "status_siglaOrgao": status_proposicao.get("siglaOrgao"),
        "status_regime": status_proposicao.get("regime"),
        "status_descricaoTramitacao": status_proposicao.get("descricaoTramitacao"),
        "status_codTipoTramitacao": status_proposicao.get("codTipoTramitacao"),
        "status_descricaoSituacao": status_proposicao.get("descricaoSituacao"),
        "status_codSituacao": status_proposicao.get("codSituacao"),
        "status_despacho": status_proposicao.get("despacho"),
        "status_ambito": status_proposicao.get("ambito"),
        "status_apreciacao": status_proposicao.get("apreciacao"),
    }

# versão "safe": não quebra o loop se algum id falhar
def safe_obter_proposicao_detalhe(id: int) -> Optional[Dict[str, Any]]:
    try:
        return obter_proposicao_detalhe(id)
    except Exception:
        return None

# aplica sobre todos os ids (com um pequeno intervalo para não sobrecarregar a API)
detalhes = []
for x in df_proposicoes_simples["id"].tolist():
    time.sleep(0.1)
    d = safe_obter_proposicao_detalhe(int(x))
    if d is not None:
        detalhes.append(d)

df_proposicoes = pd.DataFrame(detalhes)
df_proposicoes.head()


---

## 3) **Baixar** os PDFs de inteiro teor

Com os URLs dos documentos de inteiro teor, podemos fazer o download da coleção completa. Vamos criar uma função simples de download. Usamos `try/except` e uma versão “safe” para evitar quebra do código. Cada arquivo será salvo com o nome `"id" + ".pdf"`, para podermos associar às proposições por id.


In [ ]:
def baixar_inteiro_teor(id: int, url: str, diretorio: str = "inteiro_teor"):
    if not url:
        return
    Path(diretorio).mkdir(parents=True, exist_ok=True)
    destino = Path(diretorio) / f"{id}.pdf"
    try:
        r = requests.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(destino, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    except Exception:
        # falhou a request; seguimos em frente
        pass

def safe_baixar_inteiro_teor(id: int, url: str, diretorio: str = "inteiro_teor"):
    try:
        baixar_inteiro_teor(id, url, diretorio)
    except Exception:
        pass

# realiza os downloads com pequeno intervalo
outdir = "inteiro_teor"
for _, row in df_proposicoes.iterrows():
    time.sleep(0.1)
    safe_baixar_inteiro_teor(int(row["id"]), row.get("urlInteiroTeor"), outdir)

# lista de arquivos baixados (pode estar vazia se você não executou os downloads)
arquivos = sorted([p.name for p in Path(outdir).glob("*.pdf")])
len(arquivos), arquivos[:10]


---

## 4) Lendo os PDFs e montando um **corpus**

Agora a parte final da coleta dos dados: transformar os PDFs baixados em uma única tabela com **id**, **número de páginas**, **texto completo** e **tamanho** (em caracteres). Para termos um texto único por proposição, vamos "colar" os textos com `\n`.  
**Pressuposto**: os documentos têm OCR (para 2025 isso é razoável). Caso não tenham caracteres reconhecíveis, será necessário aplicar OCR antes (ex.: Tesseract) e só depois extrair o texto.


In [ ]:
def ler_pdf(arquivo_pdf: str, pasta: str = "inteiro_teor"):
    caminho = Path(pasta) / arquivo_pdf
    texto = ""
    try:
        texto = extract_text(str(caminho)) or ""
    except Exception:
        texto = ""

    # conta páginas com PyPDF2
    n_paginas = None
    try:
        reader = PdfReader(str(caminho))
        n_paginas = len(reader.pages)
    except Exception:
        n_paginas = None

    _id = arquivo_pdf.replace(".pdf", "")
    return {
        "id": _id,
        "n_paginas": n_paginas,
        "text": texto,
        "n_caracteres": len(texto),
    }

corpus_docs = pd.DataFrame([ler_pdf(a) for a in arquivos])
corpus_docs.head()


---

## 5) **(Opcional)**: já dá pra brincar com análises de texto

Abaixo vai um exemplo equivalente ao que faríamos com `tidytext`: tokenização simples, remoção de stopwords e contagem de palavras mais frequentes.


In [ ]:
stop_pt = set(stopwords.words('portuguese'))

def preprocess_text(s: str) -> str:
    # padroniza: minúsculas, remove acentos
    s = (s or "").lower()
    s = unidecode(s)
    return s

# tokenização básica por split em espaços (para um pipeline mais robusto, considere regex/Spacy)
tokens = []
for _, row in corpus_docs.iterrows():
    txt = preprocess_text(row.get("text", ""))
    words = [w for w in txt.split() if w.isalpha() and w not in stop_pt]
    tokens.extend(words)

top_palavras = (
    pd.Series(tokens, name="word")
    .value_counts()
    .head(30)
    .rename_axis("word")
    .reset_index(name="n")
)
top_palavras


> Dica: se quiser padronizar antes (minúsculas, remover pontuação, tirar acentos), o bloco `preprocess_text` já resolve boa parte; para limpeza adicional, você pode aplicar regex para remover pontuação antes de tokenizar. Isso costuma deixar o corpus mais fácil de trabalhar.
